In [1]:
# Svet scraper > XML output

from urllib.request import urlopen
import certifi
import ssl
import json
import sys
from urllib.error import HTTPError, URLError
import time

BLANK_PAGE_SZ = 6350   # Not Found Web page size
magazine = "Svet"
urlpath  = "https://pretraziva.rs/pregled/svet"
urlpath2 = "https://pretraziva.rs/prikaz/svet"
urlpath3 = "https://pretraziva.rs/download/svet--"

# Make a list of all available issues

# Get a list of all issues from the JavaScript variable "publicationYears"
str1 = urlopen(urlpath, timeout=10, context=ssl.create_default_context(cafile=certifi.where()))
htmlstr = str1.read().decode('utf-8')
i1 = htmlstr.index("{")
i2 = htmlstr.index("};")
dictionary = eval(htmlstr[i1:i2+1])

# Print the information

print("\nYears:", flush=True)
for y in dictionary:
    print(y, end=" ", flush=True)

print("\nAll issues by date:", flush=True)
for y in dictionary:
    print(y)
    issues = dictionary[y]
    for i in issues:
        print("  " + i[1].replace(" ","") + y + ".", flush=True)
print()

# Move the text of an issue to a file

# Iterate all pages by year/issue and write the plain text to new files
for y in dictionary:    # years
    
    issues = dictionary[y]

    for i in issues:    # issues
        ymd = y + "-" + i[1][4:6] + "-" + i[1][0:2]
        iurlpath = urlpath2 + "/" + ymd
        print(y, iurlpath, flush=True)
        
        # Iterate pages

        # Number of pages
        iurlpath2 = iurlpath + "/1"
        istr2 = urlopen(iurlpath2, timeout=10, context=ssl.create_default_context(cafile=certifi.where()))
        html2 = istr2.read().decode('utf-8')
        i1 = html2.index("var lastPage = ")
        lp = html2[i1+15:i1+19]
        i2 = lp.index(";")
        npg = eval(lp[:i2])
        print(npg, "pages", flush=True)
        
        # Write text to a new file
        ioutpath = ymd + " " + magazine + ".xml"
        print("Writing text to:", ioutpath, flush=True)
        f = open(ioutpath, "w", encoding='utf-8',newline='')
        
        # Add TEI header
        print('Adding header', flush=True)
        with open("TEI header.txt") as header:
            for line in header:
                f.write(line)

        for pg in range(1, npg+1):
            print("  page", pg, flush=True)
            try:
                try:
                    iurlpath3 = urlpath3 + ymd + "--" + str(pg) + ".txt"
                    str3 = urlopen(iurlpath3, timeout=10, context=ssl.create_default_context(cafile=certifi.where()))
                    txt  = str3.read().decode('utf-8')
                    f.write(txt)
                except:
                    iurlpath3 = urlpath3 + ymd + "--0" + str(pg) + ".txt"
                    str3 = urlopen(iurlpath3, timeout=10, context=ssl.create_default_context(cafile=certifi.where()))
                    txt  = str3.read().decode('utf-8')
                    f.write(txt)
            except:
                pass
        
        # Add TEI closed tags at the end
        print('Adding footer', flush=True)
        with open("TEI closed tags.txt") as footer:
            for line in footer:
                f.write(line)
    
        f.close()
        
print('Done.', flush=True)


Years:
1995 1996 1997 1999 2013 2014 
All issues by date:
1995
  17.02.1995.
  03.03.1995.
  17.03.1995.
  31.03.1995.
  14.04.1995.
  28.04.1995.
  12.05.1995.
  09.06.1995.
  23.06.1995.
  07.07.1995.
  21.07.1995.
  07.08.1995.
  21.08.1995.
  04.09.1995.
  02.10.1995.
  16.10.1995.
  30.10.1995.
  13.11.1995.
  27.11.1995.
  11.12.1995.
  25.12.1995.
1996
  08.01.1996.
  22.01.1996.
  04.02.1996.
  19.02.1996.
  04.03.1996.
  18.03.1996.
  01.04.1996.
  15.04.1996.
  29.04.1996.
  13.05.1996.
  27.05.1996.
  10.06.1996.
  24.06.1996.
  08.07.1996.
  22.07.1996.
  05.08.1996.
  19.08.1996.
  02.09.1996.
  16.09.1996.
  30.09.1996.
  14.10.1996.
  28.10.1996.
  11.11.1996.
  09.12.1996.
  23.12.1996.
1997
  06.01.1997.
  20.01.1997.
  17.02.1997.
  03.03.1997.
  17.03.1997.
  31.03.1997.
  14.04.1997.
  07.07.1997.
  21.07.1997.
1999
  01.03.1999.
  15.03.1999.
  12.04.1999.
  26.04.1999.
2013
  25.11.2013.
  02.12.2013.
  09.12.2013.
  16.12.2013.
  23.12.2013.
  30.12.2013.
2014
 

In [1]:
#Scuffed word count 

import os, glob

total = 0
filecount = 0

for filename in glob.glob('*.xml'):
    with open(os.path.join(os.getcwd(), filename), 'r') as f:
        filecount = filecount + 1
        data = f.read()
        words = data.split()
        total = total + len(words)
        
total = total - filecount*57       # 57 is the WC in the two TEI files
print(total)

3370068
